<a href="https://colab.research.google.com/github/vanderbilt-ml/50-Crook-mlproj-Honesty/blob/main/HonestyNotebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Project Overview
This project will attempt to predict a correlation between a person's religion and whether they display personality traits indicitive of Machiavellian tendencies. Included in this prediction will be whether the person considers themselves more introverted or extroverted.

## Data Set
Machivallianism Test on Kaggle

## Performance Measures
This will be based on a percentage of people with more honest traits and what their religious preference is along with whether they identify as being more reserved or extroverted.